In [1]:
# library
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from slice_ojh import base_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout

from sklearn.metrics import accuracy_score

# seed
import os
seed = 23
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
# 사진으로부터 데이터 추출
x_loc = [27]
y_loc = [710, 733]
for i in range(19):
    x_loc.append(int(x_loc[0]+(25.5*(i+1))))
print(x_loc)
print(y_loc)

data = []
for i in range(12):
    tmp = Image.open(f'baduk/baduk{i+1}.jpg')
    black = tmp.convert('L')
    pix = np.array(black)
    for x in x_loc:
        for y in y_loc:
            data.append(pix[y][x])

print(len(data))

# 이상치 제거
outlier = list(filter(lambda x: 10 < data[x] < 200, range(len(data))))
outlier = sorted(outlier, reverse=True)
print(outlier)

for r in outlier:
    del data[r]
print(len(data))

# 이진분류 데이터로 전처리
for i in range(len(data)):
    if data[i] <= 10:
        data[i] = 0 # 흑색
    if data[i] >= 200:
        data[i] = 1 # 백색
data = np.array(data)
data = data.reshape(-1, 1)
print(data)

# 데이터 분할
features, labels = base_split(data, 10, 1)
print(features.shape)
print(labels.shape)

x_train, x_test, y_train, y_test = train_test_split(features, labels,
                                                    test_size=0.05, shuffle=False)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

[27, 52, 78, 103, 129, 154, 180, 205, 231, 256, 282, 307, 333, 358, 384, 409, 435, 460, 486, 511]
[710, 733]
480
[479, 439, 359, 239, 159, 39]
474
[[0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1

In [3]:
# 모델링
def LSTM_model(drop=0.2, optimizer='adam'):
    inputs = Input(shape=(10,1))
    l1 = LSTM(64, activation='selu', return_sequences=True)(inputs)
    l1 = Dropout(drop)(l1)
    l2 = LSTM(32, activation='selu')(l1)
    l2 = Dropout(drop)(l2)
    l3 = Dense(20, activation='selu')(l2)
    l3 = Dropout(drop)(l3)
    outputs = Dense(1)(l3)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
# training
model = LSTM_model(0.2, 'adam')
history = model.fit(features, labels, batch_size=10, epochs=100)

Train on 464 samples
Epoch 1/100
464/464 [==============================] - 3s 5ms/sample - loss: 2.8389 - accuracy: 0.5237
Epoch 2/100
464/464 [==============================] - 0s 871us/sample - loss: 2.3632 - accuracy: 0.4849
Epoch 3/100
464/464 [==============================] - 0s 877us/sample - loss: 1.2122 - accuracy: 0.4634
Epoch 4/100
464/464 [==============================] - 0s 897us/sample - loss: 1.0021 - accuracy: 0.4677
Epoch 5/100
464/464 [==============================] - 0s 905us/sample - loss: 0.9007 - accuracy: 0.4828
Epoch 6/100
464/464 [==============================] - 0s 882us/sample - loss: 0.8425 - accuracy: 0.4655
Epoch 7/100
464/464 [==============================] - 0s 894us/sample - loss: 0.8714 - accuracy: 0.4698
Epoch 8/100
464/464 [==============================] - 0s 954us/sample - loss: 1.1156 - accuracy: 0.4957
Epoch 9/100
464/464 [==============================] - 0s 897us/sample - loss: 1.8774 - accuracy: 0.4806
Epoch 10/100
464/464 [==============

464/464 [==============================] - 0s 1ms/sample - loss: 0.7186 - accuracy: 0.4741
Epoch 79/100
464/464 [==============================] - 0s 988us/sample - loss: 0.6991 - accuracy: 0.5022
Epoch 80/100
464/464 [==============================] - 0s 903us/sample - loss: 0.6966 - accuracy: 0.5302
Epoch 81/100
464/464 [==============================] - 0s 975us/sample - loss: 0.7103 - accuracy: 0.5086
Epoch 82/100
464/464 [==============================] - 0s 886us/sample - loss: 0.6980 - accuracy: 0.5216
Epoch 83/100
464/464 [==============================] - 0s 927us/sample - loss: 0.7026 - accuracy: 0.5517
Epoch 84/100
464/464 [==============================] - 0s 950us/sample - loss: 0.6968 - accuracy: 0.5237
Epoch 85/100
464/464 [==============================] - 0s 881us/sample - loss: 0.7069 - accuracy: 0.4892
Epoch 86/100
464/464 [==============================] - 0s 947us/sample - loss: 0.6974 - accuracy: 0.5302
Epoch 87/100
464/464 [==============================] - 0s 92

In [5]:
x_test = tf.cast(x_test, tf.float32)
pred = model.predict(x_test)
print(pred.shape)

y_test = y_test.reshape(-1, 1)
acc = accuracy_score(pred.round(), y_test)
acc

(24, 1)


0.6666666666666666